<a href="https://colab.research.google.com/github/rajivsresearch/Corona/blob/master/Visualize_Covid19_Dataset_using_Plotly.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

To visualize the Covid 19 cases spreading across the continents using Plotly

In [0]:
import pandas as pd

import datetime as dt
import plotly.express as px

In [0]:
# Data from this website will be used for Covid 19 Cases

url = 'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv'
df = pd.read_csv(url)

In [3]:
df.head(1)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,1/28/20,1/29/20,1/30/20,1/31/20,2/1/20,2/2/20,2/3/20,2/4/20,2/5/20,2/6/20,2/7/20,2/8/20,2/9/20,2/10/20,2/11/20,2/12/20,2/13/20,2/14/20,2/15/20,2/16/20,2/17/20,2/18/20,2/19/20,2/20/20,2/21/20,2/22/20,2/23/20,2/24/20,2/25/20,2/26/20,...,4/18/20,4/19/20,4/20/20,4/21/20,4/22/20,4/23/20,4/24/20,4/25/20,4/26/20,4/27/20,4/28/20,4/29/20,4/30/20,5/1/20,5/2/20,5/3/20,5/4/20,5/5/20,5/6/20,5/7/20,5/8/20,5/9/20,5/10/20,5/11/20,5/12/20,5/13/20,5/14/20,5/15/20,5/16/20,5/17/20,5/18/20,5/19/20,5/20/20,5/21/20,5/22/20,5/23/20,5/24/20,5/25/20,5/26/20,5/27/20
0,NaN,Afghanistan,33.0,65.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,...,933,996,1026,1092,1176,1279,1351,1463,1531,1703,1828,1939,2171,2335,2469,2704,2894,3224,3392,3563,3778,4033,4402,4687,4963,5226,5639,6053,6402,6664,7072,7653,8145,8676,9216,9998,10582,11173,11831,12456


In [0]:
# Data from Plotly to get the names of the Continents
df1 = px.data.gapminder()

In [5]:
df1.head(1)

,country,continent,year,lifeExp,pop,gdpPercap,iso_alpha,iso_num
0,Afghanistan,Asia,1952,28.801,8425333,779.445314,AFG,4


In [8]:
# The Covid 19 cases for some countries like Australia and Canada are broken into provinces

df.loc[df['Country/Region']=='Australia', :]

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,1/28/20,1/29/20,1/30/20,1/31/20,2/1/20,2/2/20,2/3/20,2/4/20,2/5/20,2/6/20,2/7/20,2/8/20,2/9/20,2/10/20,2/11/20,2/12/20,2/13/20,2/14/20,2/15/20,2/16/20,2/17/20,2/18/20,2/19/20,2/20/20,2/21/20,2/22/20,2/23/20,2/24/20,2/25/20,2/26/20,...,4/18/20,4/19/20,4/20/20,4/21/20,4/22/20,4/23/20,4/24/20,4/25/20,4/26/20,4/27/20,4/28/20,4/29/20,4/30/20,5/1/20,5/2/20,5/3/20,5/4/20,5/5/20,5/6/20,5/7/20,5/8/20,5/9/20,5/10/20,5/11/20,5/12/20,5/13/20,5/14/20,5/15/20,5/16/20,5/17/20,5/18/20,5/19/20,5/20/20,5/21/20,5/22/20,5/23/20,5/24/20,5/25/20,5/26/20,5/27/20
8,Australian Capital Territory,Australia,-35.4735,149.0124,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,103,103,104,104,104,104,105,106,106,106,106,106,106,106,106,106,107,107,107,107,107,107,107,107,107,107,107,107,107,107,107,107,107,107,107,107,107,107,107,107
9,New South Wales,Australia,-33.8688,151.2093,0,0,0,0,3,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,...,2936,2957,2963,2969,2971,2976,2982,2994,3002,3004,3016,3016,3025,3030,3035,3033,3035,3042,3044,3047,3051,3053,3053,3053,3059,3063,3071,3074,3075,3076,3078,3081,3082,3084,3086,3087,3090,3092,3089,3090
10,Northern Territory,Australia,-12.4634,130.8456,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,28,28,28,28,28,28,28,28,28,28,28,28,28,27,29,29,29,29,29,29,29,29,29,29,29,29,29,29,29,29,29,29,29,29,29,29,29,29,29,29
11,Queensland,Australia,-28.0167,153.4000,0,0,0,0,0,0,0,1,3,2,3,2,2,3,3,4,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,...,1015,1019,1019,1024,1024,1026,1026,1026,1030,1033,1034,1033,1033,1034,1035,1038,1043,1043,1045,1045,1045,1045,1045,1051,1052,1051,1054,1055,1055,1057,1057,1058,1058,1058,1060,1061,1056,1057,1058,1058
12,South Australia,Australia,-34.9285,138.6007,0,0,0,0,0,0,0,0,0,0,1,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,...,435,435,435,437,438,438,438,438,438,438,438,438,438,438,438,438,438,438,438,439,439,439,439,439,439,439,439,439,439,439,439,439,439,439,439,439,439,439,440,440
13,Tasmania,Australia,-41.4545,145.9707,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,188,195,200,201,205,207,207,207,212,214,218,219,221,221,221,221,221,225,226,227,227,227,227,227,227,227,227,228,228,228,228,228,228,228,228,228,228,228,228,228
14,Victoria,Australia,-37.8136,144.9631,0,0,0,0,1,1,1,1,2,3,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,...,1319,1328,1329,1336,1336,1337,1343,1346,1349,1349,1354,1361,1364,1371,1384,1406,1423,1440,1454,1467,1468,1487,1496,1511,1514,1521,1540,1551,1558,1564,1573,1573,1581,1593,1593,1603,1605,1610,1618,1628
15,Western Australia,Australia,-31.9505,115.8605,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,544,545,545,546,546,546,548,549,549,549,550,551,551,551,551,551,551,551,551,552,552,552,552,553,553,552,552,552,553,554,557,557,557,557,557,560,560,564,570,570


In [9]:
df.loc[df['Country/Region']=='Canada', :]

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,1/28/20,1/29/20,1/30/20,1/31/20,2/1/20,2/2/20,2/3/20,2/4/20,2/5/20,2/6/20,2/7/20,2/8/20,2/9/20,2/10/20,2/11/20,2/12/20,2/13/20,2/14/20,2/15/20,2/16/20,2/17/20,2/18/20,2/19/20,2/20/20,2/21/20,2/22/20,2/23/20,2/24/20,2/25/20,2/26/20,...,4/18/20,4/19/20,4/20/20,4/21/20,4/22/20,4/23/20,4/24/20,4/25/20,4/26/20,4/27/20,4/28/20,4/29/20,4/30/20,5/1/20,5/2/20,5/3/20,5/4/20,5/5/20,5/6/20,5/7/20,5/8/20,5/9/20,5/10/20,5/11/20,5/12/20,5/13/20,5/14/20,5/15/20,5/16/20,5/17/20,5/18/20,5/19/20,5/20/20,5/21/20,5/22/20,5/23/20,5/24/20,5/25/20,5/26/20,5/27/20
35,Alberta,Canada,53.9333,-116.5765,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,2562,2803,2908,3095,3401,3720,4017,4233,4480,4696,4850,5165,5355,5573,5670,5766,5836,5893,5963,6017,6098,6157,6253,6300,6345,6407,6457,6515,6587,6644,6683,6716,6735,6768,6800,6818,6860,6879,6901,6926
36,British Columbia,Canada,49.2827,-123.1207,0,0,0,0,0,0,1,1,1,1,1,1,1,1,2,2,4,4,4,4,4,4,4,4,4,4,5,5,5,5,6,6,6,6,7,7,...,1618,1647,1647,1724,1795,1824,1853,1948,1948,1998,2053,2087,2112,2145,2171,2171,2224,2232,2255,2288,2315,2330,2330,2353,2360,2376,2392,2407,2428,2428,2444,2446,2467,2479,2507,2517,2517,2530,2541,2550
37,Grand Princess,Canada,37.6489,-122.6655,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,13,13,13,13,13,13,13,13,13,13,13,13,13,13,13,13,13,13,13,13,13,13,13,13,13,13,13,13,13,13,13,13,13,13,13,13,13,13,13,13
38,Manitoba,Canada,53.7609,-98.8139,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,253,254,254,255,257,262,263,267,271,273,273,275,277,281,282,282,283,284,286,283,284,284,287,289,290,290,289,289,289,289,290,290,290,290,292,292,292,292,292,292
39,New Brunswick,Canada,46.5653,-66.4619,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,117,118,118,118,118,118,118,118,118,118,118,118,118,118,118,118,118,119,120,120,120,120,120,120,120,120,120,120,120,120,120,120,120,121,121,121,121,121,122,123
40,Newfoundland and Labrador,Canada,53.1355,-57.6604,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,257,257,257,257,256,256,256,257,258,258,258,258,258,259,259,259,259,259,259,261,261,261,261,261,261,261,261,260,260,260,260,260,260,260,260,260,260,260,260,260
41,Nova Scotia,Canada,44.6820,-63.7443,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,649,675,721,737,772,827,850,865,873,900,915,935,947,959,963,971,985,991,998,1007,1008,1011,1018,1019,1020,1024,1026,1034,1037,1040,1043,1044,1045,1046,1048,1049,1050,1051,1052,1053
42,Ontario,Canada,51.2538,-85.3232,0,0,0,0,1,1,1,1,1,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,4,4,4,...,11013,11561,12063,12715,13718,14068,14550,15012,15568,15970,16500,16978,17395,17880,18321,18574,19097,19468,19910,20388,20826,21148,21469,21817,22158,22516,22865,23258,23645,23974,24286,24755,25197,25595,26085,26560,26897,27302,27624,27943
43,Prince Edward Island,Canada,46.5107,-63.4168,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,26,26,26,26,26,26,26,26,26,26,27,27,27,27,27,27,27,27,27,27,27,27,27,27,27,27,27,27,27,27,27,27,27,27,27,27,27,27,27,27
44,Quebec,Canada,52.9399,-73.5491,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,17521,17950,19319,20126,20965,21838,22616,23267,24109,24983,25761,26610,27550,28656,29664,31873,32631,33425,34334,35249,36161,36997,37732,38480,39235,39940,40732,41429,42192,42928,43636,44206,44784,45504,46150,46847,47420,47993,48607,49148


In [0]:
# The data for countries with provinces will need to be summed up

df2 = df.iloc[:,1:].groupby('Country/Region').sum().reset_index()

In [0]:
# Get only the countries and continents from the gapminder dataframe

df3 = df1[['country', 'continent']].drop_duplicates()

In [0]:
# Combine the 2 datasets

df4 = df2.merge(df3, how='left', left_on='Country/Region', right_on='country')

In [18]:
df4.head(1)

,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,1/28/20,1/29/20,1/30/20,1/31/20,2/1/20,2/2/20,2/3/20,2/4/20,2/5/20,2/6/20,2/7/20,2/8/20,2/9/20,2/10/20,2/11/20,2/12/20,2/13/20,2/14/20,2/15/20,2/16/20,2/17/20,2/18/20,2/19/20,2/20/20,2/21/20,2/22/20,2/23/20,2/24/20,2/25/20,2/26/20,2/27/20,...,4/20/20,4/21/20,4/22/20,4/23/20,4/24/20,4/25/20,4/26/20,4/27/20,4/28/20,4/29/20,4/30/20,5/1/20,5/2/20,5/3/20,5/4/20,5/5/20,5/6/20,5/7/20,5/8/20,5/9/20,5/10/20,5/11/20,5/12/20,5/13/20,5/14/20,5/15/20,5/16/20,5/17/20,5/18/20,5/19/20,5/20/20,5/21/20,5/22/20,5/23/20,5/24/20,5/25/20,5/26/20,5/27/20,country,continent
0,Afghanistan,33.0,65.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,...,1026,1092,1176,1279,1351,1463,1531,1703,1828,1939,2171,2335,2469,2704,2894,3224,3392,3563,3778,4033,4402,4687,4963,5226,5639,6053,6402,6664,7072,7653,8145,8676,9216,9998,10582,11173,11831,12456,Afghanistan,Asia


In [20]:
# Around 60 countries are without a continent

df4.loc[df4['continent'].isna(),'Country/Region'].count()

60

In [0]:
# Download the dataframe for manual processing

df4.to_excel('Covid Cases.xlsx')

In [0]:
# After manually inputting the continents, load the new dataset

df5 = pd.read_excel('/content/Covid Cases World.xlsx', index_col=0)

In [26]:
df5.head(1)

,Country/Region,country,continent,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,1/28/20,1/29/20,1/30/20,1/31/20,2/1/20,2/2/20,2/3/20,2/4/20,2/5/20,2/6/20,2/7/20,2/8/20,2/9/20,2/10/20,2/11/20,2/12/20,2/13/20,2/14/20,2/15/20,2/16/20,2/17/20,2/18/20,2/19/20,2/20/20,2/21/20,2/22/20,2/23/20,2/24/20,2/25/20,...,4/18/20,4/19/20,4/20/20,4/21/20,4/22/20,4/23/20,4/24/20,4/25/20,4/26/20,4/27/20,4/28/20,4/29/20,4/30/20,5/1/20,5/2/20,5/3/20,5/4/20,5/5/20,5/6/20,5/7/20,5/8/20,5/9/20,5/10/20,5/11/20,5/12/20,5/13/20,5/14/20,5/15/20,5/16/20,5/17/20,5/18/20,5/19/20,5/20/20,5/21/20,5/22/20,5/23/20,5/24/20,5/25/20,5/26/20,5/27/20
0,Afghanistan,Afghanistan,Asia,33.0,65.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,...,933,996,1026,1092,1176,1279,1351,1463,1531,1703,1828,1939,2171,2335,2469,2704,2894,3224,3392,3563,3778,4033,4402,4687,4963,5226,5639,6053,6402,6664,7072,7653,8145,8676,9216,9998,10582,11173,11831,12456


In [0]:
# Some countries are still missing continents, they need to be removed

f1 = df5['continent'].isna()

df6 = df5.loc[~f1,:]

In [0]:
# Get the date column names for the melt process

dates = df6.columns.values[5:]

In [0]:
# Unpivot the date colummns 

df7 = pd.melt(df6, id_vars=['continent', 'Country/Region'], value_vars=dates, var_name='Date')

In [41]:
df7.head()

,continent,Country/Region,Date,value
0,Asia,Afghanistan,1/22/20,0
1,Europe,Albania,1/22/20,0
2,Africa,Algeria,1/22/20,0
3,Europe,Andorra,1/22/20,0
4,Africa,Angola,1/22/20,0


In [52]:
# Plotly magic with animation!

fig = px.bar(df7, 
             x = 'continent', 
             y = 'value', 
             color = 'continent', 
             animation_frame = 'Date', 
             animation_group = 'Country/Region', 
             range_y = [0,3000000], 
             height=600, 
             title = 'Spread of Covid 19', 
             labels={'continent':'Continent', 'value':'Number of Covid Cases'}, 
             hover_name='Country/Region')
fig.show()